In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots

from ipynb.fs.full.busiest_times_and_days import *

In [3]:
orders = pd.read_csv("D:/github/Python/Python/Maven Pizza Challenge/Resources/pizza_sales/orders.csv", encoding='utf-8')                       
order_details = pd.read_csv("D:/github/Python/Python/Maven Pizza Challenge/Resources/pizza_sales/order_details.csv", encoding='utf-8')
pizza_types = pd.read_csv("D:/github/Python/Python/Maven Pizza Challenge/Resources/pizza_sales/pizza_types.csv", encoding='ISO-8859-1')
pizzas = pd.read_csv("D:/github/Python/Python/Maven Pizza Challenge/Resources/pizza_sales/pizzas.csv", encoding='utf-8')

In [4]:
orders_time = orders['time'].sort_index()                                                      
orders_count = order_details['order_id'].sort_index()
orders_date = orders['date'].sort_index()
orders_quantity = order_details['quantity']

orders_datetime = (orders_date + ' ' + orders_time).astype('datetime64')                       

frame = {'Orders Count': orders_count, 'Orders Quantity': orders_quantity}         
                                                                                                            
pizza_orders = pd.DataFrame(frame)
pizza_orders = pizza_orders.groupby('Orders Count').agg({'Orders Quantity': 'sum'}).reset_index()
pizza_orders['Datetime'] = orders_datetime

final_orders = pizza_orders.set_index('Datetime')

In [5]:
final_orders_month = final_orders.groupby(final_orders.index.month_name()).agg(
    {'Orders Count': 'count', 'Orders Quantity': 'sum'}
)

bar_pizzas_per_month = px.bar(final_orders_month, y="Orders Quantity", x=final_orders_month.index,
            labels={
                "Orders Quantity": "No. of Pizzas made",
                "Datetime": "Month"
            },
            title="Pizzas made per month (2015)", width=800, height=400,
            text=final_orders_month["Orders Quantity"],
            color=final_orders_month["Orders Quantity"] 
            )

bar_pizzas_per_month.update_traces(textposition='auto')
bar_pizzas_per_month.update_xaxes(categoryorder='array', 
    categoryarray= ['January', 'February', 
    'March', 'April','May', 
    'June','July', 'August', 
    'September','October','November',
    'December'])

In [6]:
final_orders_day = final_orders.groupby(final_orders.index.day_name()).agg(
    {'Orders Count': 'count', 'Orders Quantity': 'sum'}
) + 1

bar_pizzas_per_day = px.bar(final_orders_day, y="Orders Quantity", x=final_orders_day.index,
            labels={
                "Orders Quantity": "No. of Pizzas made",
                "Datetime": "Day"
            },
            title="Pizzas made per days of a week (2015)", width=800, height=400,
            text=final_orders_day["Orders Quantity"],
            color=final_orders_day["Orders Quantity"] 
            )

bar_pizzas_per_day.update_traces(textposition='auto')
bar_pizzas_per_day.update_xaxes(categoryorder='array', 
    categoryarray= ['Monday','Tuesday','Wednesday','Thursday', 'Friday', 'Saturday', 'Sunday'])

In [7]:
final_orders_hour = final_orders.resample("1h", label='right').agg(
    {'Orders Quantity': 'sum', 'Orders Count': 'count'} 
)

final_orders_hour = final_orders_hour.groupby(final_orders_hour.index.hour).agg(
    {'Orders Quantity': 'sum', 'Orders Count': 'sum'}
)

get_orders_hour_morning(final_orders_hour.index)
get_orders_hour_afternoon(final_orders_hour.index)
get_orders_hour_evening(final_orders_hour.index)
get_orders_hour_night(final_orders_hour.index)

final_orders_hour['Category'] = get_day_category(final_orders_hour.index)

In [8]:
bar_orders_hour = px.bar(final_orders_hour, y="Orders Quantity", x=final_orders_hour.index, 
    color=final_orders_hour["Category"],
            labels={
                "Orders Count": "No. of Orders",
                "Datetime": "Hour",
                "Category": "Time of Day"
            },
            title="Orders per hour (2015)", width=800, height=400,
            hover_data={'Category'}, 
            text=final_orders_hour["Orders Quantity"])

bar_orders_hour.update_traces(textposition='auto')
bar_orders_hour.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 5, 10, 15, 20],
        ticktext = ['12:00 a.m.', '5:00 a.m.', 
        '10:00 a.m.', '15:00 p.m.', '20:00 p.m.']
    )
)